# GCN
$$
X' = D^{0.5}AD^{-0.5}XW
$$
node-wise:
$$
x' = \sum_j{[e_{ij}/sqrt(d_i*d_j)]*(x_j·W)}
$$
## edge_weight写法参考
- https://pytorch-geometric.readthedocs.io/en/latest/notes/create_gnn.html?highlight=propagate#the-messagepassing-base-class
- +edge_weight
- https://github.com/zwt233/AIR/blob/c83b27a17f6b4fa37b70f5a1492dcdb4636a970a/src/OGB/layer.py#L138

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import degree, add_self_loops, add_remaining_self_loops

`propagate()`方法 internally(内部) calls `message()`, `aggregate()` and `update()`

In [17]:
class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr='add')  # "Add" aggregation (Step 5). #*batch node_dim默认=-2,可以适应有/无batch情况
        self.lin = nn.Linear(in_channels, out_channels, bias=False)
        self.bias = nn.Parameter(torch.Tensor(out_channels))

        self.reset_parameters()

    def reset_parameters(self):
        self.lin.reset_parameters()
        self.bias.data.zero_()

    def forward(self, x, edge_index, edge_weight = None):#* 添加可修改权重edge_weight
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]
        
        # Step 1: Add self-loops to the adjacency matrix.
        edge_index, edge_weight = add_self_loops(edge_index, edge_weight, num_nodes=x.size(-2)) #*edge_weight

        # Step 2: Linearly transform node feature matrix.
        x = self.lin(x)

        # Step 3: Compute normalization.
        row, col = edge_index
        deg = degree(col, x.size(-2), dtype=x.dtype) #*batch
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]
        norm_edge_weight = deg_inv_sqrt[row] * edge_weight * deg_inv_sqrt[col] #*edge_weight

        # Step 4-5: Start propagating messages.
        if edge_weight == None:
            out = self.propagate(edge_index, x=x, norm=norm)
        else:
            out = self.propagate(edge_index, x=x, norm=norm_edge_weight)

        # Step 6: Apply a final bias vector.
        out += self.bias

        return out

    def message(self, x_j, norm):
        # x_j has shape [E, out_channels]

        # Step 4: Normalize node features.
        return norm.view(-1, 1) * x_j

### 测试edge_weight
```
>>>norm
tensor([0.7071, 0.5000, 1.0000, 0.5000, 0.5000])
>>>norm_edge_weight
tensor([0.3536, 0.2500, 1.0000, 0.5000, 0.5000])
```

### self_loops
> edge_index = torch.tensor( [[0,1,2],
                            [1,2,2]])

```
# add_self_loops添加自环会重复
add_self_loops(edge_index, num_nodes=x.shape[0])[0]

tensor([[0, 1, 2, 0, 1, 2],
        [1, 2, 2, 0, 1, 2]])

# add_remaining_self_loops 则能排除重复
add_remaining_self_loops(edge_index, num_nodes=x.shape[0])[0]

tensor([[0, 1, 0, 1, 2],
        [1, 2, 0, 1, 2]])
```

In [10]:
x = torch.rand([1,3,2]) # B,N,C
x

tensor([[[0.6187, 0.8069],
         [0.6074, 0.8397],
         [0.1793, 0.0573]]])

In [12]:
edge_index = torch.tensor( [[0,1],
                            [1,2]])
edge_weight = torch.tensor( [0.5,0.5])
model=GCNConv(x.shape[-1],1)
model(x,edge_index,edge_weight)

tensor([[[-0.0306],
         [-0.0222],
         [-0.0236]]], grad_fn=<AddBackward0>)

In [5]:
x = torch.rand([3,3,2]) # B,N,C
x

tensor([[[0.1336, 0.4313],
         [0.4649, 0.0240],
         [0.4016, 0.6170]],

        [[0.4014, 0.8971],
         [0.5770, 0.8196],
         [0.0829, 0.2064]],

        [[0.9773, 0.4091],
         [0.4734, 0.1871],
         [0.9651, 0.6834]]])

In [6]:
edge_index = torch.tensor( [[0,1],
                            [1,2]])
edge_weight = torch.tensor( [0.5,0.5])
model=GCNConv(x.shape[-1],1)
model(x,edge_index,edge_weight)

tensor([[[-0.0247],
         [-0.1662],
         [-0.1685]],

        [[-0.1360],
         [-0.1823],
         [-0.0795]],

        [[-0.6066],
         [-0.3622],
         [-0.3517]]], grad_fn=<AddBackward0>)